In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import helpers as h
import re
from collections import Counter
import math

## Initial exploration

In [3]:
data = h.read_input()

In [4]:
data.head(n=12)

,information
0,Tile 3923:
1,...##.....
2,.##.#..#.#
3,###....#..
4,...#...#.#
5,....#.#...
6,..........
7,.....#.###
8,.##.#..#..
9,..##..#...


Each tile is given by 11 rows: 1 with the tile name and 10 with the elements

In [6]:
len(data) / 11

144.0

In total,a 12x12 image (144 titles) is saved in a file with 11*144=1584 rows

In [118]:
data["information"].iloc[0]

'Tile 3923:'

We will enumerate the tiles from 1 to 144 as they appear in the file

In [169]:
x = h.get_image(1, data)

In [170]:
x

1     ...##.....
2     .##.#..#.#
3     ###....#..
4     ...#...#.#
5     ....#.#...
6     ..........
7     .....#.###
8     .##.#..#..
9     ..##..#...
10    ..#.#.##..
Name: information, dtype: object

Each image is tranformed into a matrix of 1 and 0 to operate faster: '.' = 1 and '#' = 0

In [171]:
m=h.image_to_matrix(x)
m

array([[1, 1, 1, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
       [1, 1, 1, 1, 0, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 0, 1, 0, 0, 1, 1]])

In [172]:
x2 = h.matrix_to_image(m)

In [173]:
x2

0    ...##.....
1    .##.#..#.#
2    ###....#..
3    ...#...#.#
4    ....#.#...
5    ..........
6    .....#.###
7    .##.#..#..
8    ..##..#...
9    ..#.#.##..
Name: information, dtype: object

In [174]:
h.get_tile_number(1, data)

3923

In [125]:
# number of tiles 12 x 12 image
len(data) // 11

144

In [175]:
all_tiles = [(k, h.get_tile_number(k, data)) for k in range(1, 145)]

Each tile, can be rotated and fliped to obtain a total of different 8 matrices (dihedral group D₄: 4 rotations and 4 flips)

In [176]:
all_m = h.transform(m)

In [177]:
all_m

Original           [[1, 1, 1, 0, 0, 1, 1, 1, 1, 1], [1, 0, 0, 1, ...
90° Rotation       [[1, 1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 1, 0, 1, ...
180° Rotation      [[1, 1, 0, 0, 1, 0, 1, 0, 1, 1], [1, 1, 1, 0, ...
270° Rotation      [[1, 0, 1, 0, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, ...
Horizontal Flip    [[1, 1, 1, 1, 1, 0, 0, 1, 1, 1], [0, 1, 0, 1, ...
Vertical Flip      [[1, 1, 0, 1, 0, 1, 0, 0, 1, 1], [1, 1, 0, 0, ...
d1                 [[1, 1, 1, 0, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, ...
d2                 [[1, 1, 0, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 1, ...
dtype: object

The boundaries can be transformed into decimal values with high bit the one the most on the left or the most on the right

In [178]:
h.boundary_to_decimal(m)

[927, 695, 851, 895]

The boundary at the right (2) reads

In [129]:
m[:,-1]

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 1])

In [130]:
binary_boundary="".join(map(str, m[:,-1]))

In [131]:
int(binary_boundary, 2)

695

For a given piece, we have a total of 8 different pieces that can be generated by rotation and flipping

In [179]:
[h.boundary_to_decimal(m2) for m2 in all_m]

[[927, 695, 851, 895],
 [1019, 927, 949, 851],
 [811, 1019, 999, 949],
 [695, 811, 895, 999],
 [999, 895, 811, 695],
 [851, 949, 927, 1019],
 [949, 999, 1019, 811],
 [895, 851, 695, 927]]

In [180]:
m

array([[1, 1, 1, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
       [1, 1, 1, 1, 0, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 0, 1, 0, 0, 1, 1]])

In [134]:
''.join(map(str, m[0]))

'1110011111'

Before working with the full example (12x12), we work the 3x3 example

## Example

In [181]:
data_example = h.read_input('input_example.txt')

In [182]:
data_example['information']

0     Tile 2311:
1     ..##.#..#.
2     ##..#.....
3     #...##..#.
4     ####.#...#
         ...    
94    .#...#.##.
95    #.#####.##
96    ..#.###...
97    ..#.......
98    ..#.###...
Name: information, Length: 99, dtype: object

In [183]:
tile_numbers = [tile[5:9] for i, tile in enumerate(data_example['information']) if i % 11 ==0]

In [184]:
print(tile_numbers)

['2311', '1951', '1171', '1427', '1489', '2473', '2971', '2729', '3079']


In [185]:
[(k, h.get_tile_number(k, data_example)) for k in range(1, 10)]

[(1, 2311),
 (2, 1951),
 (3, 1171),
 (4, 1427),
 (5, 1489),
 (6, 2473),
 (7, 2971),
 (8, 2729),
 (9, 3079)]

In total, we have 9 tiles

### Boundaries

Each piece, taking into account the 6 possible transformations (4 rotations and 2 flips), can generate up to 8 different shapes, that result in 8 different boundaries also

In [205]:
n_tiles = 9
z = []
for i in range(1, n_tiles + 1):
    tile = h.get_tile_number(i, data_example)
    img = h.get_image(i, data_example)
    m = h.image_to_matrix(img)
    m_all = h.transform(m)
    for j in range(8):
        boundary = h.boundary_to_decimal(m_all.iloc[j])
        for k in range(len(boundary)):
            z.append([tile, j+1, k+1, boundary[k]])
all_boundaries = pd.DataFrame(z, columns=['id', 'transformation', 'boundary', 'value'])

In [206]:
all_boundaries = h.get_all_boundaries(data_example)

In [188]:
all_boundaries.head()

,id,transformation,boundary,value
0,2311,1,1,813
1,2311,1,2,934
2,2311,1,3,792
3,2311,1,4,525
4,2311,2,1,705


In [189]:
np.unique(all_boundaries['value'])

array([  57,   61,   75,   99,  121,  175,  182,  313,  321,  334,  343,
        407,  436,  447,  458,  459,  481,  491,  522,  525,  542,  567,
        624,  626,  632,  675,  705,  723,  735,  752,  759,  789,  792,
        813,  839,  840,  846,  862,  907,  927,  934,  938,  945,  957,
        980,  999, 1005, 1014])

In [202]:
all_boundaries_data = h.get_all_boundaries(data)
b_unique = np.unique(all_boundaries_data['value'])

In [203]:
unique_boundaries = [int(x) for x in b_unique] 

We have to prove that all boundary values are not symatrical and that all baoundary values and its reversed values are different

In [192]:
# Generate all 10-digit binary palindromes and convert them to decimal

def generate_binary_palindromes(n_bits):
    half = n_bits // 2
    palindromes = []
    for i in range(2**half):
        left = f"{i:0{half}b}"
        full = left + left[::-1]
        palindromes.append(int(full, 2))
    return palindromes

# Generate for 10-bit binary palindromes
palindromes_10_bit = generate_binary_palindromes(10)
palindromes_10_bit

[0,
 48,
 72,
 120,
 132,
 180,
 204,
 252,
 258,
 306,
 330,
 378,
 390,
 438,
 462,
 510,
 513,
 561,
 585,
 633,
 645,
 693,
 717,
 765,
 771,
 819,
 843,
 891,
 903,
 951,
 975,
 1023]

In [204]:
for x in palindromes_10_bit:
    if x in unique_boundaries:
        print(x)

All boundary values are non symmetric, that is that the boundary values and its reverse are not identical. therefore, no boundary in the exemple is symmetric.

### Pieces

Each piece can put in 8 different shapes (after combinations of 4 rotations and 2 flips)

In [194]:
all_transforms = h.get_all_transforms(data_example)

In [195]:
all_transforms

,id,transformation,boundaries
0,2311,1,"[813, 934, 792, 525]"
1,2311,2,"[705, 813, 407, 792]"
2,2311,3,"[99, 705, 723, 407]"
3,2311,4,"[934, 99, 525, 723]"
4,2311,5,"[723, 525, 99, 934]"
...,...,...,...
67,3079,4,"[759, 907, 407, 522]"
68,3079,5,"[522, 407, 907, 759]"
69,3079,6,"[839, 957, 321, 934]"
70,3079,7,"[957, 522, 934, 907]"


In [196]:
n=3
[len(Counter(all_boundaries['value'][(n*32):((n+1)*32)])) for n in range(0,9)]

[8, 8, 8, 8, 8, 8, 8, 8, 8]

In [197]:
n=0
Counter(all_boundaries['value'][range(0+n*32,(n+1)*32)])

Counter({927: 4, 695: 4, 851: 4, 895: 4, 1019: 4, 949: 4, 811: 4, 999: 4})

### Coincidences

In [207]:
all_transforms[all_transforms['id'] == 2311]

,id,transformation,boundaries
0,2311,1,"[813, 934, 792, 525]"
1,2311,2,"[705, 813, 407, 792]"
2,2311,3,"[99, 705, 723, 407]"
3,2311,4,"[934, 99, 525, 723]"
4,2311,5,"[723, 525, 99, 934]"
5,2311,6,"[792, 407, 813, 705]"
6,2311,7,"[407, 723, 705, 99]"
7,2311,8,"[525, 792, 934, 813]"


A coincidence is when two pieces share a common boundary. This function computes the number of other pieces that have the same boundary b in piece id. For instance, boundary 813 in piece 2311 is found in another piece.

In [208]:
h.n_coincidences(813, 2311, all_boundaries)

1

In [209]:
id = 2311 # tile
b = 813 # boundary value
new_df = all_boundaries[all_boundaries['id']!=id] 
new_df[new_df['value'] == b][['id','transformation', 'boundary']]


,id,transformation,boundary
98,1427,1,3
103,1427,2,4
116,1427,6,1
125,1427,8,2


Boundary 813 exists also in tile 1427 in 4 different boundary positions. Given a boundary value, and a boundary position there is only one tile shape possible. This is the key to join all pieces to create the final matrix. For all possible boundary values, we find how many other pieces have the same boundary. 

In [210]:
# all coincidence for all boundaries for each piece and boundary
all_coincidences = [h.n_coincidences(int(r[3]), int(r[0]), all_boundaries) for r in all_boundaries.values]
np.unique(all_coincidences).tolist()

[0, 1]

Each piece has a maximum of one common border with any other piece

### Number of common boundaries in a tile 

We compute it only for the tiles in the original shape. To match them, we will have to transform them

In [211]:
y = all_transforms
i_count = 0
z=[]
for tile in y.values :
    if i_count % 8 == 0:
        n_matches = 0
        for b in tile[2]:
            n_matches = n_matches + h.n_coincidences(b, tile[0], all_boundaries)
        print([tile[0], tile[1], n_matches])
        z.append([tile[0], tile[1], n_matches])
    i_count = i_count + 1

[2311, 1, 3]
[1951, 1, 2]
[1171, 1, 2]
[1427, 1, 4]
[1489, 1, 3]
[2473, 1, 3]
[2971, 1, 2]
[2729, 1, 3]
[3079, 1, 2]


In [57]:
print(z)

[[2311, 1, 3], [1951, 1, 2], [1171, 1, 2], [1427, 1, 4], [1489, 1, 3], [2473, 1, 3], [2971, 1, 2], [2729, 1, 3], [3079, 1, 2]]


The pieces with 2 common boundaries, are corners. The ones with 3 are the pieces in the boundary and the ones with 4 are the pieces inside. In this case, the corners are the pieces 1951, 1171, 2971 and 3079

In [212]:
corners = []
for x in z:
    if x[2] == 2:
        corners.append(x[0])
corners


[1951, 1171, 2971, 3079]

In [213]:
math.prod(corners)

20899048083289

This is the solution of question 1


In [214]:
data = h.read_input()
all_boundaries_data = h.get_all_boundaries(data)
all_transforms_data = h.get_all_transforms(data)

y_data = all_transforms_data
i_count = 0
z_data=[]
for tile in y_data.values :
    if i_count % 8 == 0:
        n_matches = 0
        for b in tile[2]:
            n_matches = n_matches + h.n_coincidences(b, tile[0], all_boundaries_data)
        # print([tile[0], tile[1], n_matches])
        z_data.append([tile[0], tile[1], n_matches])
    i_count = i_count + 1

In [215]:
corners_data = []
for x in z_data:
    if x[2] == 2:
        corners_data.append(x[0])
corners_data

[2243, 1213, 2953, 2273]

In [216]:
math.prod(corners_data)

18262194216271

## Second question.

We have first to build the complete image with the right pieces with the right transform in each position.

We solve it for the simple example first (3x3). Let us take the four corner tiles and identify whihc boundaries are external

In [217]:
all_boundaries[all_boundaries['id'].isin(corners)]

,id,transformation,boundary,value
32,1951,1,1,313
33,1951,1,2,525
34,1951,1,3,459
35,1951,1,4,182
36,1951,2,1,436
...,...,...,...,...
283,3079,7,4,907
284,3079,8,1,407
285,3079,8,2,839
286,3079,8,3,759


In [218]:
all_transforms[all_transforms['id'].isin(corners)]

,id,transformation,boundaries
8,1951,1,"[313, 525, 459, 182]"
9,1951,2,"[436, 313, 705, 459]"
10,1951,3,"[846, 436, 626, 705]"
11,1951,4,"[525, 846, 182, 626]"
12,1951,5,"[626, 182, 846, 525]"
13,1951,6,"[459, 705, 313, 436]"
14,1951,7,"[705, 626, 436, 846]"
15,1951,8,"[182, 459, 525, 313]"
16,1171,1,"[57, 735, 999, 121]"
17,1171,2,"[632, 57, 1005, 999]"


In [219]:
# all coincidence for all boundaries for each piece and boundary
all_boundaries_corners = all_boundaries[all_boundaries['id'].isin(corners)]
all_coincidences = [(int(r[0]),int(r[1]),int(r[2]), int(r[3]), h.n_coincidences(int(r[3]), int(r[0]), all_boundaries)) for r in all_boundaries_corners.values]
#np.unique(all_coincidences).tolist()
boundary_corner = [x for x in all_coincidences if x[4]==0]

In [220]:
boundary_corner

[(1951, 1, 3, 459, 0),
 (1951, 1, 4, 182, 0),
 (1951, 2, 1, 436, 0),
 (1951, 2, 4, 459, 0),
 (1951, 3, 1, 846, 0),
 (1951, 3, 2, 436, 0),
 (1951, 4, 2, 846, 0),
 (1951, 4, 3, 182, 0),
 (1951, 5, 2, 182, 0),
 (1951, 5, 3, 846, 0),
 (1951, 6, 1, 459, 0),
 (1951, 6, 4, 436, 0),
 (1951, 7, 3, 436, 0),
 (1951, 7, 4, 846, 0),
 (1951, 8, 1, 182, 0),
 (1951, 8, 2, 459, 0),
 (1171, 1, 3, 999, 0),
 (1171, 1, 4, 121, 0),
 (1171, 2, 1, 632, 0),
 (1171, 2, 4, 999, 0),
 (1171, 3, 1, 927, 0),
 (1171, 3, 2, 632, 0),
 (1171, 4, 2, 927, 0),
 (1171, 4, 3, 121, 0),
 (1171, 5, 2, 121, 0),
 (1171, 5, 3, 927, 0),
 (1171, 6, 1, 999, 0),
 (1171, 6, 4, 632, 0),
 (1171, 7, 3, 632, 0),
 (1171, 7, 4, 927, 0),
 (1171, 8, 1, 121, 0),
 (1171, 8, 2, 999, 0),
 (2971, 1, 1, 862, 0),
 (2971, 1, 4, 567, 0),
 (2971, 2, 1, 945, 0),
 (2971, 2, 2, 862, 0),
 (2971, 3, 2, 945, 0),
 (2971, 3, 3, 491, 0),
 (2971, 4, 3, 567, 0),
 (2971, 4, 4, 491, 0),
 (2971, 5, 1, 491, 0),
 (2971, 5, 2, 567, 0),
 (2971, 6, 3, 862, 0),
 (2971, 6, 

In [247]:
canvas=[]
direction = 'down'
searched_boundary = 1 
# First corner
# position (1, 1)
x = [1951, 2]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
n1_piece = n_piece
canvas.append(x)

In [227]:
n_piece['boundaries']

9    [436, 313, 705, 459]
Name: boundaries, dtype: object

In [248]:
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundary + 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundary)]
next_tile.iloc[0]

id                2311
transformation       2
boundary             1
value              705
Name: 4, dtype: int64

In [249]:
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
n1_piece = n_piece
canvas.append(x)

In [250]:
x

[2311, 2]

In [252]:
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundary + 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundary)]
next_tile.iloc[0]

id                3079
transformation       8
boundary             1
value              407
Name: 284, dtype: int64

In [253]:
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
n1_piece = n_piece
canvas.append(x)

In [245]:
x

[3079, 8]

In [254]:
canvas

[[1951, 2], [2311, 2], [3079, 8]]

In [263]:
direction = 'right'
searched_boundaries = [2, 4] 
# First corner
# position (1, 1)
x = canvas[0]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]


In [256]:
n_piece

,id,transformation,boundaries
9,1951,2,"[436, 313, 705, 459]"


In [264]:
n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]

313

In [267]:
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]

id                2729
transformation       2
boundary             4
value              313
Name: 231, dtype: int64

In [268]:
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)
direction = 'down'
searched_boundaries = [3, 1] 

In [269]:
canvas

[[1951, 2], [2311, 2], [3079, 8], [2729, 2]]

In [270]:
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]

id                1427
transformation       2
boundary             1
value             1014
Name: 100, dtype: int64

In [271]:
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)

In [273]:
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)

In [274]:
direction = 'right'
searched_boundaries = [2, 4]
x = canvas[3]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)

In [275]:
canvas

[[1951, 2], [2311, 2], [3079, 8], [2729, 2], [1427, 2], [2473, 3], [2971, 2]]

In [276]:
direction = 'down'
searched_boundaries = [3, 1]
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)

In [277]:
direction = 'down'
searched_boundaries = [3, 1]
next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) & (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
next_tile.iloc[0]
x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)

In [278]:
canvas

[[1951, 2],
 [2311, 2],
 [3079, 8],
 [2729, 2],
 [1427, 2],
 [2473, 3],
 [2971, 2],
 [1489, 2],
 [1171, 4]]

In [ ]:
n = 3
canvas=[]
direction = 'down'
searched_boundary = 1 
# First corner
# position (1, 1)
x_initial = [1951, 2]
n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
canvas.append(x)
for col in range(n):
    for row in range(n):
        if (col == 0) & (row == 0):
            x = x_initial
            n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
            canvas.append(x)
        else:
            next_tile = all_boundaries[(all_boundaries['value'] == n_piece.iloc[0]['boundaries'][searched_boundaries[0] - 1]) &
                                        (all_boundaries['id'] != x[0]) & (all_boundaries['boundary'] == searched_boundaries[1])]
            x = [int(next_tile.iloc[0]['id']), int(next_tile.iloc[0]['transformation'])]
            n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]
            canvas.append(x)
        direction = 'down'
        searched_boundaries = [3, 1]
    direction = 'right'
    searched_boundaries = [2, 4] 
    x = canvas[col * 3]
    n_piece = all_transforms[(all_transforms['id'] == x[0]) & (all_transforms['transformation'] == x[1])]  

IndexError: single positional indexer is out-of-bounds